In [ ]:
from scipy.optimize import linprog
import numpy as np

# Debt data: (Loan amount, Interest rate, Min payment)
debts = np.array([
    [5000, 0.15, 200],  # Loan 1: M5,000 at 15% interest, min payment M200
    [3000, 0.12, 150],  # Loan 2: M3,000 at 12% interest, min payment M150
])

# Monthly income & expenses
income = 5000  
expenses = 2500  

# Define LP problem
num_debts = len(debts)
c = debts[:, 1]  # Interest rates (minimize total interest)
A = np.vstack([-np.eye(num_debts), np.ones(num_debts)])  # Constraints for debt payment
b = np.hstack([-debts[:, 2], income - expenses])  # Payments should cover min requirements

# Solve the LP problem
result = linprog(c, A_ub=A, b_ub=b, method="highs")
optimal_payments = result.x  

print("Optimized Debt Payments:", optimal_payments)


In [7]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np

# Sample Data
data = pd.DataFrame({
    "income": [5000, 6000, 4000, 3000, 7000],
    "expenses": [2500, 3000, 2200, 2700, 3500],
    "debt": [5000, 2000, 7000, 1000, 4000],
    "savings": [500, 1000, 300, 200, 1200],
    "label": ["Stable", "Stable", "At Risk", "Critical", "Stable"]
})

# Split features & labels
X = data.drop(columns=["label"])
y = data["label"]

# Train Decision Tree
model = DecisionTreeClassifier()
model.fit(X, y)

# Predict Financial Status
new_user = np.array([[3000, 3200, 2500, 800]])  # Sample user data
prediction = model.predict(new_user)
print("Financial Status:", prediction)


Financial Status: ['Critical']


c:\Users\Administrator\Desktop\projects\thabo motlakase\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [5]:
import numpy as np
from scipy.optimize import linprog

# Debt Data: (Loan amount, Interest rate (monthly), Min payment, Max payment)
debts = np.array([
    [5000, 0.15/12, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12/12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
])

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses = 2500  # Fixed monthly expenses
months = 12  # Planning period
num_debts = len(debts)

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, num_debts))
remaining_balances = debts[:, 0].copy()

def prioritize_debts(strategy="avalanche"):
    """Returns the sorted index of debts based on the chosen strategy."""
    if strategy == "avalanche":  # Highest interest rate first
        return np.argsort(-debts[:, 1])
    elif strategy == "snowball":  # Lowest balance first
        return np.argsort(debts[:, 0])
    return np.arange(num_debts)

priority_order = prioritize_debts(strategy="avalanche")  # Change to "snowball" if needed

for month in range(months):
    available_cash = income_variability[month] - expenses  # Calculate available cash
    c = debts[:, 1]  # Minimize total interest
    A = np.vstack([-np.eye(num_debts), np.ones(num_debts)])  # Payment constraints
    b = np.hstack([-debts[:, 2], available_cash])  # Min payment & budget limit
    
    # Solve LP for optimal payments
    result = linprog(c, A_ub=A, b_ub=b, bounds=[(debts[i, 2], debts[i, 3]) for i in range(num_debts)], method="highs")
    payments[month, :] = result.x if result.success else debts[:, 2]  # Use min payments if LP fails
    
    # Update remaining balances & compound interest for unpaid balances
    for i in priority_order:
        remaining_balances[i] = max(0, remaining_balances[i] - payments[month, i])
        remaining_balances[i] += remaining_balances[i] * debts[i, 1]  # Apply interest

# Print final optimized payments
print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balances After", months, "Months:")
print(remaining_balances)



Optimized Monthly Payments:
[[200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]]

Remaining Balances After 12 Months:
[3199.5494015 1459.0758839]


In [8]:
import numpy as np
from scipy.optimize import linprog

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses = 2500  # Fixed monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.15, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.18, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.10, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
available_cash = np.mean(income_variability) - expenses

def recommend_loan():
    best_loan = None
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        total_interest = loan[1] * loan[0] * months  # Approximate total interest over period
        score = total_interest / max_affordable_payment  # Lower is better
        
        if score < best_score:
            best_score = score
            best_loan = loan
    
    return best_loan

recommended_loan = recommend_loan()
print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_loan[0]:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, 1))
remaining_balance = recommended_loan[0]
interest_rate = recommended_loan[1]

for month in range(months):
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", months, "Months:")
print(remaining_balance)


Recommended Loan Based on Income & Expenses:
Amount: M4000.00, Interest Rate: 10.00%, Min Payment: M180.00, Max Payment: M800.00

Optimized Monthly Payments:
[[800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]]

Remaining Balance After 12 Months:
0.0


In [10]:
import numpy as np
from scipy.optimize import linprog

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses_variability = [2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400]  # Monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.10, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.30, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.20, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.25, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
average_income = np.mean(income_variability)
average_expenses = np.mean(expenses_variability)
available_cash = average_income - average_expenses

def recommend_loan():
    best_loan = None
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        total_interest = loan[1] * loan[0] * months  # Approximate total interest over period
        score = total_interest / max_affordable_payment  # Lower is better
        
        if score < best_score:
            best_score = score
            best_loan = loan
    
    return best_loan

recommended_loan = recommend_loan()
print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_loan[0]:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, 1))
remaining_balance = recommended_loan[0]
interest_rate = recommended_loan[1]

for month in range(months):
    available_cash = income_variability[month] - expenses_variability[month]  # Adjust per month
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", months, "Months:")
print(remaining_balance)



Recommended Loan Based on Income & Expenses:
Amount: M5000.00, Interest Rate: 10.00%, Min Payment: M200.00, Max Payment: M1000.00

Optimized Monthly Payments:
[[1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]]

Remaining Balance After 12 Months:
0.0


In [11]:
import numpy as np

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses_variability = [2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400]  # Monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.15, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.18, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.10, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
average_income = np.mean(income_variability)
average_expenses = np.mean(expenses_variability)
available_cash = average_income - average_expenses

def recommend_loan():
    best_loan = None
    best_amount = 0
    best_months = 0
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        for term in range(6, 61, 6):  # Loan terms from 6 to 60 months
            monthly_interest = loan[1]
            loan_amount = min(loan[0], max_affordable_payment * term)  # Determine max loan amount
            
            total_interest = 0
            balance = loan_amount
            for _ in range(term):
                interest = balance * monthly_interest
                total_interest += interest
                balance -= max_affordable_payment
                balance += interest  # Add interest to balance
                if balance <= 0:
                    break
            
            score = total_interest / term  # Lower is better
            
            if score < best_score:
                best_score = score
                best_loan = loan
                best_amount = loan_amount
                best_months = term
    
    return best_loan, best_amount, best_months

recommended_loan, recommended_amount, recommended_months = recommend_loan()

print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_amount:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}, Repayment Period: {recommended_months} months")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((recommended_months, 1))
remaining_balance = recommended_amount
interest_rate = recommended_loan[1]

for month in range(recommended_months):
    available_cash = income_variability[month % len(income_variability)] - expenses_variability[month % len(expenses_variability)]  # Adjust per month
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", recommended_months, "Months:")
print(remaining_balance)


Recommended Loan Based on Income & Expenses:
Amount: M3000.00, Interest Rate: 12.00%, Min Payment: M150.00, Max Payment: M700.00, Repayment Period: 60 months

Optimized Monthly Payments:
[[700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]]

Remaining Balance After 60 Months:
0.0
